# Computer Lab 6: Automating Scientific Discovery

Welcome to the practical session of [Data-Driven Life Sciences course module 6](https://ddls.aicell.io/course/ddls-2025/module-6/lab/).

In this lab, you will explore **automating scientific discovery and AI agents**, learning how to design, build, and evaluate agentic workflows that plan experiments, orchestrate tools and data pipelines, and interface with lab automation. You'll work hands-on with the Opentrons API for laboratory robotics simulation and develop AI agents using Model Context Protocol (MCP) tools that can autonomously design and execute experiments.

## Virtual Lab Setup

We will build a **virtual laboratory** equipped with automated liquid handling robots to simulate real wetlab operations. Our primary tool is the **Opentrons Flex**, a state-of-the-art liquid handling robot that can perform precise pipetting, sample preparation, and protocol execution. The ultimate goal is to develop **AI agents that can autonomously design and control wetlab operations**, bridging the gap between computational intelligence and physical laboratory automation.

<figure>
<center>
<img src="https://il-biosystems.com/app/uploads/2025/01/1._Opentrons_FLEX_HERO.png.png" width="350">
</figure>

*The Opentrons Flex liquid handling robot, see it in action in this [video](https://www.youtube.com/watch?v=d6ln-0LeT8A).*

## Learning Objectives

By the end of this lab, you will:
- **Master Opentrons API**: Write and simulate robotic pipetting protocols
- **Build MCP Tools**: Create remote-accessible tools using the Hypha platform  
- **Work with AI Agents**: Configure Gemini CLI to design experiments autonomously
- **Bridge Theory & Practice**: Apply coding concepts to real laboratory automation

This lab builds on concepts from the two lectures this week:

- **Wei Ouyang** (KTH Royal Institute of Technology): AI systems for next-generation life science, focusing on building intelligent laboratory automation systems
- **Gabriel Reder** (Cambridge University): Accelerating laboratory scientific discovery with AI, exploring how generative AI and large language models can automate repetitive tasks in biological research

We will use cutting-edge concepts including lab automation robotics, Hypha for distributed computing, and MCP tools to connect AI agents with laboratory equipment, transforming how experimental science is conducted.

---

**Acknowledgements:** This notebook was created by Gabriel Reder (gk@reder.io) with modifications by Wei Ouyang, and teaching assistants Songtao Cheng and Nils Mechtel.

## Environment setup

### Installing dependencies
We'll need the `opentrons` python package to simulate running protocols, the `mcp` package to create MCP tools, and `hypha-rpc` and `pydantic` to connect your MCP tool to Gemini CLI.

In [ ]:
!pip install opentrons mcp pydantic hypha-rpc

### Working with Gemini CLI in Google Colab (via VS Code tunnel)

Open the terminal in Google Colab and run the following commands to install Gemini CLI:

```bash
curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.40.3/install.sh | bash
source /root/.bashrc
nvm install 21
nvm use 21
npm install -g @google/gemini-cli
```

**Start Gemini CLI**

To start a Gemini chat session, ensure your working directory is set to your Module 6 folder in Google Drive:

```bash
cd /content/drive/MyDrive/DDLS-Course/Module6/
```

and then run the `gemini` command:

```bash
gemini
```

**Save Your Gemini Chat History**

Remember to regularly save your Gemini chat history to avoid losing records of your work. In the Gemini CLI, run:
```
/chat save computer-lab-6
```

**Copy the Checkpoint File**

Ask Gemini to copy the checkpoint file from the temporary Gemini directory to your Google Drive folder while you are still in the Gemini CLI session:
```
Run this command: `cp /root/.gemini/tmp/*/checkpoint-computer-lab-6.json .`
```

You can do so after every small or large step of your work. This will prevent the loss of your chat history if runtime in Google Colab disconnects and gets reset.

If you have already closed the Gemini CLI session, run this command to copy the checkpoint file to your Google Drive folder:
```bash
cd /content/drive/MyDrive/DDLS-Course/Module6/
cp /root/.gemini/tmp/*/checkpoint-computer-lab-6.json .
```

**Load Your Gemini Chat History**

To load your previously saved Gemini chat history, run the following command in the Gemini CLI:
```
/chat load computer-lab-6
```

---

# Creating robotic lab protocols

## A sample Opentron protocol

One thing that's nice about Opentrons is that they have a Python API, meaning we can write scripts to execute protocols directly on the robot. Take a look at the sample protocol below. Can you tell what's going on?

In [ ]:
%%capture

from opentrons.simulate import get_protocol_api


protocol = get_protocol_api(
    version="2.19", robot_type="Flex"
)

plate = protocol.load_labware(
    load_name="corning_96_wellplate_360ul_flat", location="D1"
)
tiprack_1 = protocol.load_labware(
    load_name="opentrons_flex_96_tiprack_200ul", location="D2"
)
trash = protocol.load_trash_bin(location="A3")

pipette = protocol.load_instrument(
    instrument_name="flex_1channel_1000",
    mount="left",
    tip_racks=[tiprack_1],
)

pipette.pick_up_tip(location=tiprack_1)
pipette.aspirate(volume=100, location=plate["A1"])
pipette.dispense(volume=100, location=plate["B1"])
pipette.drop_tip(location=trash)

### Tasks Introduction:

- 🌞 <font color='orange'>**Exercise**:</font> Sections marked with a 🌞 symbol introduce an exercise or question. Please read these sections carefully to understand the concepts and tasks involved.

- **⭐ Your Answer:** Cells marked with a ⭐ symbol indicate where you need to write your answer. Please provide your code or answer there.

🌞 <font color='orange'>**Exercise**:</font>

**Note**: Do these exercises manually, do *not* use your AI agent yet.

1. What do you think the protocol above is doing? Write a step-by-step plain text protocol that corresponds to what you think the protocol does.

2. Now familiarize yourself with the opentron [API documentation](https://docs.opentrons.com/v2/) and pay special attention to:
   - [Hardware Modules](https://docs.opentrons.com/v2/new_modules.html#) 
   - [Labware Library](https://labware.opentrons.com/)
   - [Deck Slots](https://docs.opentrons.com/v2/deck_slots.html)
   
   What is the difference between `Hardware` and `Labware` in the context of the opentron API?

3. Looking at the `location` designations in the script (e.g. `D1`, `D2`, `A3` vs `A1`, `B1`), explain:
   - What do the location designations like `D1` and `D2` correspond to?
   - What do the location designations like `A1` and `B1` correspond to? 
   - What is the difference between these two types of location references?

4. Find the API names for the following pieces of hardware/labware:
   - Thermocycler module
   - Magnetic block
   - GEB 96 Tip Rack 10 µL
   - Thermo Scientific Nunc 96 Well Plate 1300 µL




`⭐ Edit this cell to write your answers below:`


1.

2.

3.

4.


---

## Simulating protocols

The opentron API allows us to simulate protocols, even if we don't have a robot. Let's try this out on our sample protocol. Make sure you have executed the cell above containing the sample protocol.

Now, execute the cell below to simulate the sample protocol and see what it does.

Does the simulation match your expectations from question 1 above?

In [ ]:
for command in protocol.commands():
    print(command)

🌞 <font color='orange'>**Exercise**:</font>

You've been given a 96-well plate where column 1 contains 300µL of sample in each well of the column. You want to perform a serial dilution at 1:10 dilution factors across each column. So column 1 will contain the sample, column 2 will contain the sample diluted at 1:10, column 3 will be another 1:10 dilution (so 1:100 from the original sample) etc. You have the following working parameters for the protocol:

  - You are diluting the sample into water. You have a reservoir of water available to you on the opentron.
  - You want the final dilution column to be a 1:1000000 dilution from the original sample.
  - Use version 2.19 of the opentron API
  - Use an 8 channel pipette.
  - You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
  - Use the same set of pipette tips for the entire dilution. So you only have to pick up tips once and drop them once over the course of the entire protocol.


Use a combination of Gemini CLI (or another AI tool) and the opentron API documentation to write a protocol that will perform this serial dilution on an opentron Flex using version 2.19 of the API. Gemini CLI should be somewhat familiar with the opentron API and should be able to write opentron python code. However, you may want to give it an example script to give hints about the specific structure and syntax you are looking to generate. You may also have to revise your script by hand using a combination of Gemini CLI and the API documentation.

**The protocol must simulate successfully without errors.**

Tips for using Gemini CLI:
- If you don't want Gemini to mess with your current notebook, you can prompt it to write the code in a separate file, e.g. `serial_dilution.py`. Once the code is generated and you are happy with it, you can import your protocol into this notebook with `from serial_dilution import protocol` and then run the simulation cell below.

⭐ Write your answer in the code cell below. If it runs without errors, use the simulation cell below that to try the protocol.

In [ ]:
from opentrons.simulate import get_protocol_api

# Create a ProtocolContext for the Flex robot
protocol = get_protocol_api(
    version="2.19", robot_type="Flex"  # API version 2.19  # Flex robot
)
print("Protocol API version:", protocol.api_version)


# ⭐ Load labware
plate =
tiprack =
reservoir =
trash =

# ⭐ Load pipette
pipette =

# ⭐ Variables
dilution_factor =
num_of_dilutions =
sample_volume =
diluent_volume =

# Pickup the pipette tips
pipette.pick_up_tip()

# Add diluent to columns 2 through 7 (270 µL in each well across all rows)
for col in range(2, num_of_dilutions + 2):
    # ⭐

# Perform serial dilution across all rows in columns 1 to 6
for col in range(1, num_of_dilutions + 1):
    # ⭐ Transfer sample from current column to the next


    # ⭐ Mix to ensure proper dilution


# ⭐ Optional: Dispose of remaining sample from the last column to avoid overflow


# Dispose of the tips
pipette.drop_tip()

**Simulate the protocol:**

In [ ]:
for command in protocol.commands():
    print(command)

---

## Coding Agents with Gemini CLI

In previous exercises you have worked with Gemini CLI and even built your own MCP tools. Each tool was designed for a specific purpose, for example, one tool to move a pipette, another to measure liquid, etc. This time, we take a different approach: instead of many small tools, you will get **one powerful coding agent**.

A coding agent is an AI assistant that can write and run code step by step while remembering the current state. This means it can share the same Python environment across multiple calls. For example, if you define a variable or create a function, the agent can use it again later without redefining it. This is very different from traditional one-shot tools, which start from scratch each time.

### One Tool, Many Tasks

Normally, MCP tools are built for a single action. But with a coding agent, you only need **one tool**:

```python
@mcp.tool
def run_python(code):
    exec(code, {"pipette": pipette})
```

Here, the tool accepts any Python code as input and executes it. Because the execution environment has access to objects like `pipette`, you can write different code snippets to control simulated labware, from aspirating liquid, to dispensing, to automating whole protocols. The same tool can therefore handle a wide range of tasks.


### The `exec` Function in Python

The key to making this possible is Python’s built-in `exec` function.

`exec` takes a string of Python code and runs it as if you had typed it directly into the program.

For example:

In [ ]:
code = """
x = 5
print(x * 2)
"""

exec(code)

You can also give `exec` an **environment** (a dictionary of variables and objects) where the code will run:

In [ ]:
x = 5

code = "print(x * 2)"

exec(code, {"x": x})

This means any code you pass in can directly use objects without having to create it again. In your case, the environment includes the labware and intruments you have already set up, like the `plate`, `tiprack`, `pipette` or `trash`.

For example:

In [ ]:
%%capture

# Create a ProtocolContext for the Flex robot
protocol = get_protocol_api(version="2.19", robot_type="Flex")

# Load the labware (plate, tiprack, trash)
plate = protocol.load_labware(
    load_name="corning_96_wellplate_360ul_flat", location="D1"
)
tiprack_1 = protocol.load_labware(
    load_name="opentrons_flex_96_tiprack_200ul", location="D2"
)
trash = protocol.load_trash_bin(location="D3")

# Load the instrument (pipette)
pipette = protocol.load_instrument(
    instrument_name="flex_1channel_1000",
    mount="left",
    tip_racks=[tiprack_1],
)

# Construct a dictionary with required objects
environment = {
    "plate": plate,
    "tiprack_1": tiprack_1,
    "trash": trash,
    "pipette": pipette,
}

This environment can then be passed to `exec` to allow the executed code to access these objects directly:

In [ ]:
def run_python(code):
    exec(
        # Execute the provided Python code
        code,
        # Provide access to the protocol objects
        environment,
    )

# Example usage: run a simple pipetting command
code = """
pipette.pick_up_tip()
pipette.aspirate(10, plate["A1"])
pipette.dispense(10, plate["A2"])
pipette.drop_tip()
"""

run_python(code)

The code block you see above is a string containing Python code. This can also be generated by Gemini CLI.

Let's check if the protocol can be simulated successfully:

In [ ]:
for command in protocol.commands():
    print(command)

### Why Coding Agents?

With this approach, you no longer need to design a separate MCP tool for every single lab action. Instead:

* The coding agent uses the `run_python` tool as a **universal interface**.
* You can focus on **how to communicate with the agent**: writing clear instructions (in `Gemini.md`) and guiding it step by step to complete lab automation tasks.
* The agent can remember and build on previous steps, which makes it powerful for scripting longer workflows.


👉 In this lab, your task will be to design good instructions for the agent and practice telling it what code to run in order to automate robotic lab protocols.

# Automating a real-life protocol with Coding Agent

Now we're going to explore how we can use the combination of lab robotics and LLMs to (partly) reproduce a real-world protocol.

Our protocol will come from this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3185615/) testing the effects of various antifungal compounds on the formation of yeast biofilms.

We will see how we can speed the process of experimentation using the tools we've been learning about.

Watch the entire protocol [video](https://drive.google.com/file/d/1quQ0KedkQvKponQhSgTolfWSIwhk-gD8/view?usp=drive_link) to understand the purpose of the protocol and the steps involved.

🌞 <font color='orange'>**Exercise 1**:</font>

Say you've performed the protocol by hand up until the point of setting up the plate with yeast for biofilm formation (2:49 in the video). You now have your yeast culture prepared and a 96-well plate ready to use.

Choose the lab equipment you'll have to use from the opentron hardware/labware lists and use appropriate prompts to Gemini CLI to perform the protocol steps in the video outlined from 2:49 to 3:06 using an opentron Flex. You can make the following assumptions:

- You have prepared 80mL of diluted yeast sample and have transferred it to an appropriate reservoir.
- At the end of this script run, you'll lid the plate and parafilm it by hand.
- Use version 2.19 of the opentron API.
- Use an 8 channel pipette.
- You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
- Use the same set of pipette tips for the entire dilution. So you only have to pick up tips once and drop them once over the course of the entire protocol.

Simulate the protocol and make sure it runs correctly.

### Step 1: Define the lab equipment

In the code cell below, define the labware and instruments you will use for this task. You can refer to the [opentron labware library](https://labware.opentrons.com/) and [hardware modules](https://docs.opentrons.com/v2/new_modules.html#) to find the appropriate definitions.

The function `register_mcp_service` will then register your MCP tool with Hypha so that Gemini CLI can access it remotely. It will also keep the cell running until you stop it manually (`Runtime` -> `Interrupt Execution`). Keep it running until you have completed this exercise with Gemini CLI.

In [ ]:
from opentrons.simulate import get_protocol_api

# Setup Opentrons protocol and labware/instrument objects
protocol = get_protocol_api(version="2.19", robot_type="Flex")

# ⭐ Load labware
plate =
tiprack_1 =
trash =
# Define other labware as needed

# ⭐ Load instruments
pipette =
# Define other instruments as needed

environment = {
    "protocol": protocol,
    "plate": plate,
    "tiprack_1": tiprack_1,
    "trash": trash,
    "pipette": pipette,
    # ⭐ Add other labware/instruments here
}

### Step 2. Register your MCP tool with Hypha


[Hypha](https://hypha.aicell.io/) is our platform that enables remote access to MCP tools. With Hypha, you can:

-   Deploy MCP tools as remote services accessible via HTTP

-   Connect AI agents (like Gemini CLI) to your tools from anywhere

-   Share tools with others without complex setup

In this lab, you’ll use Hypha to make your Opentrons MCP tool available remotely, allowing Gemini CLI to access it and execute robotic protocols from any location.

---

Below you will find the full implementation of the code interpreter. Run the cell below to define the required functions and register your MCP tool with Hypha.

In [ ]:
import ast
import io
import traceback
from contextlib import redirect_stderr, redirect_stdout
from typing import Any, Dict

from hypha_rpc import connect_to_server
from hypha_rpc.utils.schema import schema_function
from pydantic import Field

# Global variable to store lab objects
ENVIRONMENT = {}

line_separator = "=" * 60 + "\n"


@schema_function
def run_python(
    code: str = Field(..., description="Python source code to execute.")
) -> str:
    """
    Execute Python code and return combined stdout, stderr, result, or error as a single string.
    Useful for print-style outputs where we want to avoid JSON encoding issues.
    """
    # Print the code being executed
    print("Executing code:")
    print("```python")
    print(code)
    print("```\n")

    # Parse the code to ensure it's valid Python
    try:
        tree = ast.parse(code, mode="exec")
    except SyntaxError as e:
        error_msg = f"SyntaxError: {e}"
        print(error_msg)
        print("\n" + line_separator)
        return error_msg

    # Detect last expression for REPL-style return
    capture_result = False
    if tree.body and isinstance(tree.body[-1], ast.Expr):
        capture_result = True
        last_expr = tree.body[-1].value
        tree.body[-1] = ast.Assign(
            targets=[ast.Name(id="_ai_result", ctx=ast.Store())],
            value=last_expr,
        )
        ast.fix_missing_locations(tree)

    code_obj = compile(tree, filename="<agent-code>", mode="exec")

    # Add lab objects to environment
    environment = {} if ENVIRONMENT is None else ENVIRONMENT.copy()

    # Execute the code and capture output
    f_stdout, f_stderr = io.StringIO(), io.StringIO()
    try:
        with redirect_stdout(f_stdout), redirect_stderr(f_stderr):
            exec(code_obj, environment)

        out = f_stdout.getvalue()
        err = f_stderr.getvalue()
        result = environment.get("_ai_result") if capture_result else None

        # Build the complete output string
        output = ""
        if out:
            output += f"Output:\n{out}\n"
        if err:
            output += f"Stderr:\n{err}\n"
        if result is not None:
            output += f"Result: {result!r}\n"

        # Print the output
        print(output)
        print(line_separator)
        return output
    except Exception as e:
        error_msg = f"Error: {type(e).__name__}\n{traceback.format_exc()}"
        print(error_msg)
        print("\n" + line_separator)
        return error_msg


async def register_mcp_service(environment: Dict[str, Any]):
    """
    Register the Python code execution service with Hypha.
    """
    global ENVIRONMENT
    ENVIRONMENT = environment

    # Register the service with Hypha
    server = await connect_to_server({"server_url": "https://hypha.aicell.io"})
    svc = await server.register_service(
        {
            "id": "python-interpreter",
            "name": "Python Interpreter",
            "description": "Execute Python code; captures stdout, stderr, and last expression result.",
            "config": {
                # Make the service public so it can be used by anyone
                "visibility": "public"
            },
            # Register the function directly (not wrapped with partial)
            "run_python": run_python,
        }
    )

    print(
        "You can test the service function using the URL below. Make sure to copy the full URL!"
    )
    print(
        f"https://hypha.aicell.io/{server.config.workspace}/services/{svc.id.split('/')[1]}/run_python?code=print('hi')"
    )

    print(line_separator)
    print("Copy this MCP service URL to your Gemini `settings.json`:\n")
    print(
        f"https://hypha.aicell.io/{server.config.workspace}/mcp/{svc.id.split('/')[1]}/mcp"
    )
    print(line_separator)

In [ ]:
# Register the service with the current environment of lab objects
await register_mcp_service(environment)

### Step 3: Add the MCP tool to Gemini CLI

Create a new file named `settings.json` and add an `mcpServers` entry like this:

```json
{
    "mcpServers": {
        "opentrons-mcp": {
            "httpUrl": "your MCP service URL"
        }
    }
}
```

Then cp this file to the Gemini config directory. WARNING: This will overwrite any existing settings.json file, so back it up if needed.

```bash
mkdir -p /root/.gemini
cp /content/drive/MyDrive/DDLS-Course/Module6/settings.json /root/.gemini/settings.json
```

Tips: Gemini CLI discovers MCP servers from this file. Use `/mcp list` in the chat to list available tools.

### 4: Write the `GEMINI.md` file

Create a new file named `GEMINI.md` in your Google Drive folder `/content/drive/MyDrive/DDLS-Course/Module6/`.

This file should describe the overall plan for the agent, including **Context**, **Goal** and **Instructions**, for example:

- The role of the agent (designing Opentrons experiments).
- What lab equipment is available and as which variables:
  - `pipette`: 8-channel P300 pipette
  - `plate`: 96-well plate
  - `reservoir`: reservoir with yeast sample
- Instructions for how to use the lab equipment. Check the equipment status before using it.
- Guardrails (no large prints, no external commands).
- Example snippets (e.g. serial dilution, column dosing).


### Step 5: Start Gemini CLI

In order for Gemini CLI to discover your `GEMINI.md` file and MCP server, you need to start a new Gemini CLI session.

Note: If you are already in Gemini CLI, save your chat history (`/chat save computer-lab-6`), then instruct Gemini to copy checkpoint to your Drive ("Run this command: `cp /root/.gemini/tmp/*/checkpoint-computer-lab-6.json .`") and exit Gemini CLI (`/quit`).

Ensure you are in the correct folder:
```bash
cd /content/drive/MyDrive/DDLS-Course/Module6/
```

Then start a new Gemini CLI session:
```bash
gemini
```

If you saved a previous chat session, you can continue that session by loading the saved history `/chat load computer-lab-6`.

Make sure the `GEMINI.md` file and the MCP server are discovered by Gemini CLI. You should see this on top of the input box in Gemini CLI: `Using: 1 GEMINI.md file | 1 MCP server (ctrl+t to view)`

To check if all your MCP tool is available, enter the command `/mcp list` in the chat interface. You should see a list of all your configured MCP tools.

Now that everything is set up, you can instruct the agent to design the experiment described in the exercise above. Make sure the agent calls your MCP tool to perform the task.

Tips for debugging:
- Most lab equipments have a state, e.g. how much liquid is in each well. You can ask the agent to print out the state of the lab equipment at any time to check if it is as expected.
- If you run into issues with the lab equipment, you can re-define it by re-running the cell in **Step 1**. This will reset the state of the lab equipment.

---

🌞 <font color='orange'>**Exercise 2 (Optional)**:</font>

To work on this task, you will need to terminate the cell above where you registered your MCP tool with Hypha (`Runtime` -> `Interrupt Execution`). Once you have stopped the cell, your previous MCP tool will no longer be available in Gemini CLI.

Now say you've continued the protocol, performing the incubation and washes by hand. You have prepared a stock solution of 1,024 µg/mL fluconazole and a stock solution of buffered RPMI 1640 medium.

Choose the lab equipment you'll have to use from the opentron hardware/labware lists and use appropriate prompts to Gemini CLI to perform the protocol steps for Antifungal Susceptibility Testing of Biofilms (3:57 to 5:04). You can make the following assumptions:

- You have prepared 80mL of fluconazole and 80mL of media and have transferred them to appropriate reservoirs.
- At the end of this script run, you'll lid the plate and parafilm it by hand.
- Use version 2.19 of the opentron API.
- Use an 8 channel pipette.
- You can order the opentron to aspirate/dispense from the first (top) well of a column using the 8 channel pipette and it will do so for the entire column. E.g. aspirating from well "A1" with an 8 channel pipette will aspirate from all wells in column 1.
- Use a *different* set of pipette tips for (1) dispensing the antifungal (2) dispensing the medium (3) performing the dilutions. So you should use a total of three set of tips (pick up / drop off operations) over the course of the protocol.

Simulate the protocol and make sure it runs correctly.

### Step 1: Define the lab equipment

In [ ]:
from opentrons.simulate import get_protocol_api

# Setup Opentrons protocol and labware/instrument objects
protocol = get_protocol_api(version="2.19", robot_type="Flex")

# ⭐ Load labware
plate =
tiprack_1 =
trash =
# Define other labware as needed

# ⭐ Load instruments
pipette =
# Define other instruments as needed

lab_objects = {
    "protocol": protocol,
    "plate": plate,
    "tiprack_1": tiprack_1,
    "trash": trash,
    "pipette": pipette,
    # ⭐ Add other labware/instruments here
}

await register_mcp_service(lab_objects)

### Step 2: Add your new MCP tool to Gemini CLI

Update the `httpUrl` in your `settings.json` file with the new URL you get when you register your MCP tool with Hypha.

```json
{
    "mcpServers": {
        "opentrons-mcp": {
            "httpUrl": "your new MCP service URL"
        }
    }
}
```

Then copy this file to the Gemini config directory. This will overwrite the previous settings.json file.

```bash
cp /content/drive/MyDrive/DDLS-Course/Module6/settings.json /root/.gemini/settings.json
```

### Step 3: Update your `GEMINI.md` file

If needed, update your `GEMINI.md` file to reflect any changes in the lab equipment or instructions for this new task.

### Step 4: Start Gemini CLI

As described in Exercise 1, start a new Gemini CLI session and ensure your new MCP tool and `GEMINI.md` file are discovered.

🌞 <font color='orange'>**Final Discussion Exercises**:</font>

1. What were the benefits of using Gemini CLI for writing these protocols? What was it able to do successfully and how much did you need to help it?

2. Look at the entire protocol from the paper [video](https://drive.google.com/file/d/1quQ0KedkQvKponQhSgTolfWSIwhk-gD8/view?usp=drive_link) again. What additional portions do you think you would be able to automate on the opentron? What parts could you automate with more advanced robotic hardware? What parts (if any) do you think can't be reliably done using robotics?


`⭐ Edit this cell to write your answers below:`


1.

2.